In [6]:
import pandas as pd
import pickle
import numpy as np

### create some sample data

In [42]:
d = {'col1': [1, 2], 'col2': [3, 4]}
mydf = pd.DataFrame(d)
newdf = pd.DataFrame(np.arange(1,100000), columns=['id'])
newdf['col1'] = 1
finaldf = pd.merge(mydf,newdf,on='col1')
finaldf.to_pickle('mydf100krows.pk')

### Downloaded the pk file and uploaded to blob storage. then i will use the file dataset to load the file back> 
Next few lines of code is just using azure ml sdk to get a pointer to registered file dataset. we will then use the mounted path to read the file.

In [3]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'bd04922c-a444-43dc-892f-74d5090f8a9a'
resource_group = 'mlplayarearg'
workspace_name = 'testdeployment'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='somedataroot')


In [35]:
import tempfile
mounted_path = tempfile.mkdtemp()
print(mounted_path)
# mount dataset onto the mounted_path of a Linux-based compute
mount_context = dataset.mount(mounted_path)

mount_context.start()

import os
print(os.listdir(mounted_path))

/tmp/tmp6547btg6
['20201020', 'FamilyFriendlyAndAdulttour.pdf', 'acquariumsticket.pdf', 'mydata.csv', 'mydf.pk', 'mydf100krows.pk', 'sample_string1.csv', 'sample_string1', 'testdataatroot.csv', 'testdataatroot2.csv', 'testfile.txt']


### Read the pk file.

In [40]:
mydfloadedback = pd.read_pickle(os.path.join(mounted_path,'mydf100krows.pk'))

In [41]:
mydfloadedback


,col1,col2,id
0,1,3,1
1,1,3,2
2,1,3,3
3,1,3,4
4,1,3,5
5,1,3,6
6,1,3,7
7,1,3,8
8,1,3,9
9,1,3,10


In [11]:
from azureml.core import Dataset, Datastore


dataset = Dataset.get_by_name(workspace, name='testpickle')

### Create dataset on the fly pointing to just one pickle file and then loading it

In [13]:
datastore = Datastore(workspace, name='newdatastoremanish')
datastore_paths = [(datastore, 'somedata/mydf100krows.pk')]
somedata = Dataset.File.from_files(path=datastore_paths)


In [14]:
import tempfile
mounted_path = tempfile.mkdtemp()
print(mounted_path)
# mount dataset onto the mounted_path of a Linux-based compute
mount_context = somedata.mount(mounted_path)

mount_context.start()

import os
print(os.listdir(mounted_path))

/tmp/tmpcgpo_v6m
['mydf100krows.pk']


In [16]:
mydata =  pd.read_pickle(os.path.join(mounted_path,os.listdir(mounted_path)[0]))

In [17]:
mydata

,col1,col2,id
0,1,3,1
1,1,3,2
2,1,3,3
3,1,3,4
4,1,3,5
5,1,3,6
6,1,3,7
7,1,3,8
8,1,3,9
9,1,3,10
